# First Attemp to create a fault visualization in 3D

## Import libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import re
from bs4 import BeautifulSoup
import tifffile as tiff
from geotiff import GeoTiff
from folium.plugins import MousePosition

## Creating the map

In [18]:
i_coords = [-33.47, -70.655]
site_map = folium.Map(location=i_coords, zoom_start=10, tiles='Stamen Terrain')

Mouse positioning

In [19]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position);

## Importing DEM

In [7]:
filename = 'DEM_Stgo.tif'
tfile = tiff.imread(filename)
tiff.imshow(tfile) # display the map

(<Figure size 988.8x604.8 with 2 Axes>,
 <AxesSubplot:>,
 <matplotlib.image.AxesImage at 0x24698efbf10>)

## Extracting Lat, Long and Elev from DEM

In [8]:
tiff_file = 'DEM_Stgo.tif'
DEM = GeoTiff(tiff_file)
zar_array = GeoTiff.read(DEM)
elev = np.array(zar_array)
lon_array, lat_array = DEM.get_coord_arrays()

In [9]:
#Raster to xyz
lon = []; lat = []; z = []

for j in range(len(elev)):
    for i in range(len(elev)):
        lon.append(lon_array[j,i])
        lat.append(lat_array[j,i])
        z.append(elev[j,i])

## Creating a Dataframe

In [10]:
if len(lon)==len(lat) & len(z)==len(lat):
    print ('Data OK!')
    print ('Creating dataframe')
    df = pd.DataFrame({'Latitud': lat, 'Longitud': lon, 'Elevation': z})
    df = df.astype('float')
    print ('Dataframe Created!')
else:
    print('Check Data')

Data OK!
Creating dataframe
Dataframe Created!


In [7]:
df.shape

(12967201, 3)

In [11]:
df['type'] = str('DEM')

In [48]:
# Dataframe to txt file
df.to_csv('DEM.txt',sep=',',index=False)
print('txt file created!')

txt file created!


## Ploting Topography

### Example Surface Plot

%matplotlib widget \
import matplotlib.pyplot as plt\
from mpl_toolkits.mplot3d import Axes3D\
import numpy as np\


X = np.linspace(-5,5,50)\
Y = np.linspace(-5,5,50)\
X, Y = np.meshgrid(X,Y)


fig = plt.figure()\
ax = fig.add_subplot(projection='3d')\
ax.plot_surface(X, Y, Z, cmap="plasma")\
plt.show()

### Extract and Drop some data to work 

In [7]:
# work well untill 10^5*
def xyz(n):
    x = []; y = []; z = []
    index = np.linspace(0, len(df)-1, n, endpoint=True, dtype='int')
    for i in index:
        x.append(df['Latitud'][i])
        y.append(df['Longitud'][i])
        z.append(df['Elevation'][i])
    return x,y,z

In [45]:
lat_m,lon_m,z_m=xyz(10**4)

### Scatter Plot

In [46]:
def scatt_plot(x,y,z):
    %matplotlib widget
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x, y, z,c=z)
    plt.show()

In [47]:
scatt_plot(lat_m,lon_m,z_m)

### Surface Plott

In [21]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')
fig_=ax.plot_surface(lon_array, lat_array, elev, cmap='gist_earth')
plt.colorbar(fig_)
plt.show()

In [14]:
def tranform_degree_to_meters(array): #for two dimensions
    m = 111120*(array-np.min(array))
    return m

In [15]:
lon_m = tranform_degree_to_meters(lon_array)
lat_m = tranform_degree_to_meters(lat_array)

In [16]:
lon_m.shape

(3601, 3601)

## Fault Trace

### Import Fault from KML

In [12]:
with open('FSR.kml','r') as file:
    data = file.read()
    
soup = BeautifulSoup (data)

coords = []
for t in soup.find_all('coordinates'):
        coords.append(t.get_text("|", strip=True))

C:\Users\vicen\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [78]:
lat_f = []; lon_f = []; trace = []

for i in range(len(coords)):
    a = coords[i].split(' ')
    for j in range(len(a)):
        b = a[j].split(',')
        lon_f.append(float(b[0]))
        lat_f.append(float(b[1]))
        trace.append(i)

In [79]:
df_fault = pd.DataFrame({'Longitud': lon_f, 'Latitud': lat_f, 'Elevation':np.zeros(len(lat_f)), 'type':'Fault', 
                         'trace':trace})

### Print Fault on the map

In [80]:
for t in range(max(df_fault.trace)+1):
    fault_points = []
    df_trace = df_fault[df_fault.trace.eq(t)]
    
    for i in range(len(df_trace)):
        fault_points.append([df_trace.Latitud.iloc[i], df_trace.Longitud.iloc[i]])
        poly_fault = folium.PolyLine(locations=fault_points, color='black').add_to(site_map)

In [81]:
site_map

### Set Elevation

In [126]:
# Array of dem points
dem_points = []
for i in range(len (elev)):
    dem_points.append((lat[i],lon[i],z[i]))

In [ ]:
# Array of fault points


In [ ]:
def distances(points):
    